In [1]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
plt.style.use('seaborn')
from scipy.stats import norm

In [2]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

In [3]:
def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors


In [4]:
# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        self.optimizer = nk.optimizer.Sgd(learning_rate=0.05)
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=0.5) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.variational.MCState(self.sampler, self.machine, n_samples=1000, n_discard=100) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output):
        self.vs.init_parameters(nk.nn.initializers.normal(stddev=0.25))
        gs = nk.VMC(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, sr=self.sr)
        # Start timing
        start = time.time()
        # Set the output files as well as number of iterations in the descent
        gs.run(out=output, n_iter=1000, show_progress = False)
        end = time.time()
        runTime = end - start
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state, runTime

In [5]:
# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err(state, edState, eng, edEng,N):
    engErr = np.abs(eng - edEng)
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return engErr, waveFunctionErr


# Combines all steps into a function to run on the cluster
def runDescentCS(N,B,Ak,alpha):
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam(N,B,Ak)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state, runTime = rbm("2021_summer_data/test_data/Logs"+str(N)) #where _call_ will be invoked
    return eng, state, runTime

In [ ]:
for i in range(21):  #here put N-1 (maximum)
    N = i+2
    alpha = 1   #density of RBM
    M = alpha*N
    # Constant A
    B = 0.95
    # Variable A
    #B=N/2
    #A = N/2
    #N0 = N/2
    # List of Ak
    Ak = []
    for i in range(N - 1):
        # Constant A
        Ak_i = 1
        # Variable A
        #Ak_i = A / (N0) * np.exp(-i / N0)
        Ak.append(Ak_i)
    # Define hamiltonian and hilbert space
    ha, hi = CSHam(N,B,Ak)

    #Exact Diagonalization
    e, v = exactDiagonalization(ha)
    #Ground state energy
    edEng = e[0]
    # Ground state
    edState = v[0]

    #Lists for Histogram Data
    numRuns = 3
    hisIt = np.arange(numRuns)
    engErr = []
    stateErr = []
    runTime = []
    

    
    
    # Get errors for each run in histogram
    for i in range(len(hisIt)):
        engTemp, stateTemp, runTimeTemp = runDescentCS(N,B,Ak,alpha)
        #engTemp, stateTemp, runTimeTemp = resultsSR[i]
        runTime.append(runTimeTemp)
        errSR = err(np.asmatrix(stateTemp), edState, engTemp, edEng,N) #make state vector as matrix data-type
        engErr.append(errSR[0])
        stateErr.append(errSR[1])
        print('Eng error ', engErr)
        print('State error ', stateErr)

    #Save data to JSON file
    data = [engErr, stateErr, runTime]
    fileName = "2021_summer_data/compromise/con_N"+str(N)+"M" + str(M)+".json"
    open(fileName, "w").close()
    with open(fileName, 'a') as file:
        for item in data:
            line = json.dumps(item)
            file.write(line + '\n')
    print('SAVED')


/home/victor1/ENV/lib/python3.8/site-packages/netket/driver/vmc.py:82: FutureWarning: The `sr` keyword argument is deprecated in favour of `preconditioner`.Please update your code to `VMC(.., precondioner=your_sr)`
  warn_deprecation(
100%|██████████| 1000/1000 [00:05<00:00, 167.14it/s, Energy=-0.93965594+0.00000023j ± 0.00000011 [σ²=0.00000000, R̂=1.4086]]
/home/victor1/ENV/lib/python3.8/site-packages/netket/driver/vmc.py:82: FutureWarning: The `sr` keyword argument is deprecated in favour of `preconditioner`.Please update your code to `VMC(.., precondioner=your_sr)`
  warn_deprecation(
  2%|▏         | 16/1000 [00:00<00:06, 150.30it/s, Energy=-0.006-0.001j ± 0.015 [σ²=0.215, R̂=1.0010]]

Eng error  [2.2574238423000992e-07]
State error  [3.290825231094807e-05]


  2%|▏         | 16/1000 [00:00<00:06, 152.80it/s, Energy=0.201+0.004j ± 0.016 [σ²=0.244, R̂=1.0024]]

Eng error  [2.2574238423000992e-07, 0.0024657058264668796]
State error  [3.290825231094807e-05, 0.0004926628645821607]


100%|██████████| 1000/1000 [00:05<00:00, 170.22it/s, Energy=-0.9380-0.0000j ± 0.0017 [σ²=0.0027, R̂=0.9987]]           


...

  0%|          | 0/1000 [00:00<?, ?it/s]

Eng error  [0.9586943100153489]
State error  [0.9999999696475023]


  0%|          | 0/1000 [00:00<?, ?it/s]

Eng error  [0.9586943100153489, 0.026700076381761306]
State error  [0.9999999696475023, 0.015159506156176228]


100%|██████████| 1000/1000 [04:13<00:00,  3.95it/s, Energy=-5.622-0.000j ± 0.019 [σ²=0.354, R̂=0.9992]]       


In [ ]:
import os
#check current directory
working_directory = os.getcwd()
print(working_directory)

In [6]:



N = 3
alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
#B=N/2
A = N/2
N0 = N/2
# List of Ak
Ak = []
for i in range(N - 1):
    # Constant A
    Ak_i = 1
    # Variable A
    #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)

#Exact Diagonalization
e, v = exactDiagonalization(ha)
#Ground state energy
edEng = e[0]
# Ground state
edState = v[0]

#Lists for Histogram Data
numRuns = 5
hisIt = np.arange(numRuns)
engErr = []
stateErr = []
runTime = []

# Cluster multiproccessing
ncpus = int(os.environ.get('SLURM_CPUS_PER_TASK',default=10))
pool = mp.Pool(processes=ncpus)
# Run Descent
results = []
for j in hisIt:
    result = pool.apply_async(runDescentCS, args=(N,B,Ak,alpha)) 
    results.append(result)

pool.close()
pool.join()
resultsSR = [r.get() for r in results]
    



# Get errors for each run in histogram
for i in range(len(hisIt)):
    #engTemp, stateTemp, runTimeTemp = runDescentCS(N,B,Ak,alpha)
    engTemp, stateTemp, runTimeTemp = resultsSR[i]
    runTime.append(runTimeTemp)
    errSR = err(np.asmatrix(stateTemp), edState, engTemp, edEng,N) #make state vector as matrix data-type
    engErr.append(errSR[0])
    stateErr.append(errSR[1])
    print('Eng error ', engErr)
    print('State error ', stateErr)

#Save data to JSON file
data = [engErr, stateErr, runTime]
fileName = "2021_summer_data/N"+str(N)+"M" + str(M)+".json"
open(fileName, "w").close()
with open(fileName, 'a') as file:
    for item in data:
        line = json.dumps(item)
        file.write(line + '\n')
print('SAVED')

/home/victor1/ENV/lib/python3.8/site-packages/netket/driver/vmc.py:82: FutureWarning: The `sr` keyword argument is deprecated in favour of `preconditioner`.Please update your code to `VMC(.., precondioner=your_sr)`
  warn_deprecation(
/home/victor1/ENV/lib/python3.8/site-packages/netket/driver/vmc.py:82: FutureWarning: The `sr` keyword argument is deprecated in favour of `preconditioner`.Please update your code to `VMC(.., precondioner=your_sr)`
  warn_deprecation(
/home/victor1/ENV/lib/python3.8/site-packages/netket/driver/vmc.py:82: FutureWarning: The `sr` keyword argument is deprecated in favour of `preconditioner`.Please update your code to `VMC(.., precondioner=your_sr)`
  warn_deprecation(
/home/victor1/ENV/lib/python3.8/site-packages/netket/driver/vmc.py:82: FutureWarning: The `sr` keyword argument is deprecated in favour of `preconditioner`.Please update your code to `VMC(.., precondioner=your_sr)`
  warn_deprecation(
/home/victor1/ENV/lib/python3.8/site-packages/netket/driver/

Eng error  [0.0022287900072177624]
State error  [0.007509688885194099]
Eng error  [0.0022287900072177624, 0.0022287900072177624]
State error  [0.007509688885194099, 0.007509688885194099]
Eng error  [0.0022287900072177624, 0.0022287900072177624, 0.0022287900072177624]
State error  [0.007509688885194099, 0.007509688885194099, 0.007509688885194099]
Eng error  [0.0022287900072177624, 0.0022287900072177624, 0.0022287900072177624, 0.0022287900072177624]
State error  [0.007509688885194099, 0.007509688885194099, 0.007509688885194099, 0.007509688885194099]
Eng error  [0.0022287900072177624, 0.0022287900072177624, 0.0022287900072177624, 0.0022287900072177624, 0.0022287900072177624]
State error  [0.007509688885194099, 0.007509688885194099, 0.007509688885194099, 0.007509688885194099, 0.007509688885194099]
SAVED
